# Implementation of the Cyberemotions agent-based modeling framework

## Exploration of model data

### Data import

First, all model results are appended one by one into one constant dataframe.

In [1]:
import glob
import pandas as pd

In [2]:
FILES = glob.glob("data/cyberemotions*.feather")

list_of_files = FILES
dataset = pd.DataFrame()

for file in list_of_files:
    df = pd.read_feather(file)
    dataset = dataset.append(df, ignore_index=True)

DATA = dataset

### Data overview

The following tables give an overview of the resulting dataframe.

In [3]:
DATA.head()

,Run,Agents,s,c,Step,v,a,A,N,h
0,0.0,100.0,0.11,0.6,0.0,0.08,-0.15,100.0,0.0,0.00
1,0.0,100.0,0.11,0.6,1.0,0.05,-0.06,90.0,0.0,0.00
2,0.0,100.0,0.11,0.6,2.0,-0.09,-0.01,81.0,3.0,0.33
3,0.0,100.0,0.11,0.6,3.0,-0.09,0.13,74.0,5.0,0.21
4,0.0,100.0,0.11,0.6,4.0,-0.03,-0.37,69.0,5.0,-0.05


In [4]:
DATA.tail()

,Run,Agents,s,c,Step,v,a,A,N,h
566880,99.0,100.0,0.2,1.5,28.0,0.25,-0.55,4.0,0.0,0.03
566881,99.0,100.0,0.2,1.5,29.0,NaN,NaN,3.0,0.0,0.01
566882,99.0,100.0,0.2,1.5,30.0,NaN,NaN,2.0,0.0,0.00
566883,99.0,100.0,0.2,1.5,31.0,NaN,NaN,1.0,0.0,0.00
566884,99.0,100.0,0.2,1.5,32.0,NaN,NaN,0.0,0.0,0.00


Since model output was uniform dataframes, they were easily combined into one large dataframe countaining roughly 567,000 rows with the following 10 columns:

| Run | Agents | s | c | Step | v | a | A | N | h |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Model run | Initial agents | Impact parameter | Satiation parameter | Timestep | Agent valence | Agent arousal | Active agents | Expressions | Field charge |

### Data extraction

The parameters of interest are impact `s` and satiation `c`, each may take 10 different values.

In [5]:
data = DATA
s_values = DATA["s"].unique()
c_values = DATA["c"].unique()
runs = DATA["Run"].unique()
print("List of all values for s = " + str(s_values))
print("List of all values for c = " + str(c_values))
print("Times each setting was run = " + str(len(runs)))

List of all values for s = [0.11 0.12 0.13 0.14 0.15 0.16 0.17 0.18 0.19 0.2 ]
List of all values for c = [0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5]
Times each setting was run = 100


Defining a function the returns a dataframe containing totals of all expressions for every possible combination of two parameters and for each model run.

In [6]:
def expressions_for_parameters(dataset, a_string, b_string, a_list, b_list, runs):
    df = pd.DataFrame()
    for a in a_list:
        df0 = dataset.loc[dataset[a_string] == a]
        for b in b_list:
            df1 = df0.loc[df0[b_string] == b]
            for run in runs:
                df2 = df1.loc[df1["Run"] == run]
                df = df.append({"N" : df2["N"].sum(), a_string : a, b_string : b}, ignore_index=True)
    return df

Applying this function to find the number of expressions `N` for each combination of `s` and `c`, storing the result in a dataframe.

In [7]:
df = expressions_for_parameters(data, "s", "c", s_values, c_values, runs)

In [8]:
df.head()

,N,c,s
0,1501.0,0.6,0.11
1,1167.0,0.6,0.11
2,1380.0,0.6,0.11
3,1054.0,0.6,0.11
4,1266.0,0.6,0.11


In [9]:
df.tail()

,N,c,s
9995,7.0,1.5,0.2
9996,310.0,1.5,0.2
9997,455.0,1.5,0.2
9998,254.0,1.5,0.2
9999,245.0,1.5,0.2


Writing the data to a `.csv` file in order to import it into a data visualization tool.

In [10]:
df.to_csv("cyberemotions-dataset-ncs.csv", index=False)